In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
from PositionwiseFFN import PositionwiseFFN
from DotProductAttention import DotProductAttention, MASKS
from MultiHeadAttention import MultiHeadAttention
from Encoder import EncoderBlock
from Decoder import DecoderBlock
from Transformer import Transformer

In [4]:
# Test PFFN
d_model=64
ffn = PositionwiseFFN(d_model)
x = torch.rand(32, 10, d_model)

assert ffn(x).shape == x.shape


In [5]:
# Test dot queries = torch.normal(0, 1, (2, 1, 2))
### Test code from d2l.ai
queries = torch.normal(0, 1, (2, 10, 2))
keys = torch.normal(0, 1, (2, 10, 2))
values = torch.normal(0, 1, (2, 10, 4))
valid_lens = torch.tensor([2, 6])

attention = DotProductAttention(dropout=0.5)
attention.eval()
assert attention(queries, keys, values).shape == (2, 10, 4)


In [6]:
# Test MHA
d_model, num_heads, inlen = 100, 5, 20
attention = MultiHeadAttention(d_model, d_model, d_model, num_heads)
batch_size = 2

X = torch.ones((batch_size, inlen, d_model))

assert attention(X, X, X, "None").shape == (batch_size, inlen, d_model)
assert attention(X, X, X, "future").shape == (batch_size, inlen, d_model)

In [7]:
# Visualize shaping of attention and masks
q = torch.rand((2,5,8))
v = torch.rand((2,5,8))
k = q

K = q.shape[1]  # Sequence length

dk = q.shape[1]**.5 
I = (q@k.transpose(1,2))

future_mask = torch.triu(torch.ones((K, K)), diagonal=1).bool()
I = I.masked_fill(future_mask, float('-inf'))
a = nn.functional.softmax(I/dk, dim=-1)@v 
#print(f"I = {I}")
#print(f"Attention = {a}")


In [8]:
# Test Encoder
dmodel=64
e = EncoderBlock(dmodel, dmodel, dmodel, h=8, dropout=0.2)
x = torch.rand(32, 10, dmodel)
assert e(x).shape == x.shape


In [9]:
# Test Decoder
dmodel=64
q = v = 10
e = DecoderBlock(dmodel, dmodel, dmodel, h=8, dropout=0.2)
x = torch.rand(32, 10, dmodel)
m = torch.rand(32, 10, dmodel)
assert e(x, m).shape == x.shape

In [5]:
# Test the Decoder 

batch_size = 32
d_model = 64  
N_WORKERS = 0

d_input = 10
d_output = 7+3*5+5  
q = 8
v = 8
h = 8
Ne = Nd = 10

optimizePrime = Transformer(d_input, d_output, d_model, q, v, h, Ne, Nd)
x = torch.rand(batch_size, 1, d_input)
assert optimizePrime(x).shape == (batch_size, 1, d_output)


In [7]:
def process_input(x)->np.array([]):
    """Takes input of shape H_thermal and concatenates the normalized vectors (age, AHe, Aft, ZHe, Zft)
    Output is shape d_input. 
    """
    x = np.array([*x])[1:,:]
    std = np.std(x, axis=1, keepdims=True)
    std[std==0] = .00001
    x = ((x - np.mean(x, axis=1, keepdims=True)) / std).T
    return x


def formattedLoader(d_input, d_output, dataset_path):
    """Returns data formatted specifically for transformer"""
    dataset = np.load(dataset_path,allow_pickle=True)
    formatted_data = []
    for x, y in dataset:
        x = process_input(x)
        y = process_output(y, d_output)
        formatted_data.append((x,y))
    return formatted_data

def rawLoader(dataset_path: str)->np.array([]):
    return np.load(dataset_path,allow_pickle=True)

In [9]:
dataset = np.load('../../Dataset/CC10k.npy',allow_pickle=True)


ModuleNotFoundError: No module named 'Sample'